### Testing how to Run Ollama locally

In [5]:
from langchain.chat_models import ChatOllama
from langchain.schema import HumanMessage

In [9]:
ollama_llm = "llama2"

chat_model = ChatOllama(
    model=ollama_llm,
)

In [10]:
messages = [HumanMessage(content="Tell me about the history of AI")]

chat_model(messages)

AIMessage(content='\nThe history of artificial intelligence (AI) dates back to ancient civilizations, where myths and legends tell stories of machines and computers that could think and act like humans. However, the modern field of AI began in the mid-20th century, when computer scientists like Alan Turing, Marvin Minsky, and John McCarthy started exploring ways to create machines that could simulate human intelligence. Here\'s a brief overview of the history of AI:\n\n1. Early Years (1950s-1960s): The term "Artificial Intelligence" was coined in 1956 by John McCarthy, and the field began to take shape with the establishment of the first AI research laboratory at Dartmouth College in 1957. Early AI research focused on developing rule-based systems, logic reasoning, and problem-solving algorithms.\n2. Rule-Based Expert Systems (1970s-1980s): The development of rule-based expert systems, such as Mycin and ELIZA, marked a significant milestone in AI history. These systems could reason and